In [4]:
from mhr.utils.utils import process_jsonl


high_resource="en ru de zh ja fr es pt".split()
low_resource="uk bg tr ar ko".split()
languages = high_resource + low_resource
metric_list=["CHAIR", "Cover", "Cog", "Ha_p", "Language_Qualified"]
metric_dict={
    "CHAIR":dict(large_is_better=False),
    "Cover":dict(large_is_better=True),
    "Cog":dict(large_is_better=False),
    "Ha_p":dict(large_is_better=False),
    "Language_Qualified":dict(large_is_better=True),
}
model_dict={
    "LLaVA Baseline":dict(
        file_path="/mnt/petrelfs/songmingyang/code/tools/lmms-eval/lmms_eval/other_evals/AMBER/model_inference/scripts/eval/origin_amber_result.jsonl",
        start_str=f"& Baseline ",
        metric_log={a:[] for a in metric_list}
        
        
    ),
    "LLaVA Ours":dict(
        file_path="/mnt/petrelfs/songmingyang/code/tools/lmms-eval/lmms_eval/other_evals/AMBER/model_inference/scripts/eval/ours_amber_result.jsonl",
        start_str=f"& Ours ",
        metric_log={a:[] for a in metric_list}
        
    ),
    
}
model_num = len(model_dict)
for model,model_item in  model_dict.items():
    model_item["result"] = process_jsonl(model_item["file_path"])

In [5]:
res = ""
metric_log = {model:{metric:[] for metric in metric_dict.keys()} for model in model_dict.keys()}
for lang_idx,lang in enumerate(high_resource):
    compare_list = {metric:[model_item["result"][lang_idx][metric] for _,model_item in model_dict.items()] for metric in metric_dict.keys()}
    lang_begin_str = f"\multirow{{{len(model_dict)}}}{{*}}{{\\centering \\textbf{{{lang}}}}}"
    res += lang_begin_str
    for model,model_item in model_dict.items():
        model_res = model_item["start_str"]
        assert lang == model_item["result"][lang_idx]["language"] ,f"lang:{lang}, res_lang:{model_item['result'][lang_idx]['language']} ,{lang_idx}"
        for metric in metric_list:
            num_res = model_item["result"][lang_idx][metric]
            metric_log[model][metric].append(num_res)
            if num_res == max(compare_list[metric]) and metric_dict[metric]["large_is_better"]:
                model_res += f"& \\textbf{{{num_res:.1f}}}"
            elif num_res == min(compare_list[metric]) and not metric_dict[metric]["large_is_better"]:
                model_res += f"& \\textbf{{{num_res:.1f}}}"
            else:
                model_res += f"& {num_res:.1f}"
        model_res += "\\\\\n"
        res += model_res
    res += "\hline\n"

avg_begin_str = f"\multirow{{{len(model_dict)}}}{{*}}{{\\centering \\textbf{{Avg$^\\text{{H}}$.}}}}"
res += avg_begin_str
avg_compare_dict = {metric:[sum(metric_log[model][metric])/len(metric_log[model][metric]) for model in model_dict.keys()] for metric in metric_dict.keys()}
for model,model_item in model_dict.items():
    model_res = model_item["start_str"].split("&")[-1]
    model_res = f"& \\rowcolor{{gray!20}} "+model_res
    for metric in metric_list:
        num_res = sum(metric_log[model][metric])/len(metric_log[model][metric])
        if num_res == max(avg_compare_dict[metric]) and metric_dict[metric]["large_is_better"]:
            model_res += f"& \\textbf{{{num_res:.1f}}}"
        elif num_res == min(avg_compare_dict[metric]) and not metric_dict[metric]["large_is_better"]:
            model_res += f"& \\textbf{{{num_res:.1f}}}"
        else:
            model_res += f"& {num_res:.1f}"
    model_res += "\\\\\n"
    res += model_res
res += "\hline\n"
    
    
metric_log = {model:{metric:[] for metric in metric_dict.keys()} for model in model_dict.keys()}
for lang_idx,lang in enumerate(low_resource):
    
    lang_idx+=len(high_resource)
    compare_list = {metric:[model_item["result"][lang_idx][metric] for _,model_item in model_dict.items()] for metric in metric_dict.keys()}
    lang_begin_str = f"\multirow{{{len(model_dict)}}}{{*}}{{\\centering \\textbf{{{lang}}}}}"
    res += lang_begin_str
    for model,model_item in model_dict.items():
        model_res = model_item["start_str"]
        assert lang == model_item["result"][lang_idx]["language"] ,f"lang:{lang}, res_lang:{model_item['result'][lang_idx]['language']}"
        for metric in metric_list:
            num_res = model_item["result"][lang_idx][metric]
            metric_log[model][metric].append(num_res)
            if num_res == max(compare_list[metric]) and metric_dict[metric]["large_is_better"]:
                model_res += f"& \\textbf{{{num_res:.1f}}}"
            elif num_res == min(compare_list[metric]) and not metric_dict[metric]["large_is_better"]:
                model_res += f"& \\textbf{{{num_res:.1f}}}"
            else:
                model_res += f"& {num_res:.1f}"
        model_res += "\\\\\n"
        res += model_res 
    res += "\hline\n"

avg_begin_str = f"\multirow{{{len(model_dict)}}}{{*}}{{\\centering \\textbf{{Avg$^\\text{{L}}$.}}}}"
res += avg_begin_str
avg_compare_dict = {metric:[sum(metric_log[model][metric])/len(metric_log[model][metric]) for model in model_dict.keys()] for metric in metric_dict.keys()}
for model,model_item in model_dict.items():
    model_res = model_item["start_str"].split("&")[-1]
    model_res = f"& \\rowcolor{{gray!20}} "+model_res
    for metric in metric_list:
        num_res = sum(metric_log[model][metric])/len(metric_log[model][metric])
        if num_res == max(avg_compare_dict[metric]) and metric_dict[metric]["large_is_better"]:
            model_res += f"& \\textbf{{{num_res:.1f}}}"
        elif num_res == min(avg_compare_dict[metric]) and not metric_dict[metric]["large_is_better"]:
            model_res += f"& \\textbf{{{num_res:.1f}}}"
        else:
            model_res += f"& {num_res:.1f}"
    model_res += "\\\\\n"
    res += model_res
res += "\hline\n"


avg_dict = {model:[[model_dict[model]["result"][lang_idx][metric] for lang_idx in range(len(languages))] for metric in metric_list]  for model in model_dict.keys()}
for avg_model,avg_model_item in avg_dict.items():
    avg_model_item = [sum(avg_model_item[metric_idx])/len(avg_model_item[metric_idx]) for metric_idx in range(len(avg_model_item))]
    avg_dict[avg_model] = avg_model_item
avg_begin_str = f"\multirow{{{len(model_dict)}}}{{*}}{{\\centering \\textbf{{Avg$^\\text{{A}}$.}}}}"
res += avg_begin_str
avg_conmpare_dict = {metric:[avg_dict[model][metric_list.index(metric)] for model in model_dict.keys()] for metric in metric_dict.keys()}
for model,model_item in model_dict.items():
    model_res = model_item["start_str"].split("&")[-1]
    model_res = f"& \\rowcolor{{gray!20}} "+model_res
    for metric in metric_list:
        num_res = avg_dict[model][metric_list.index(metric)]
        if num_res == max(avg_conmpare_dict[metric]) and metric_dict[metric]["large_is_better"]:
            model_res += f"& \\textbf{{{num_res:.1f}}}"
        elif num_res == min(avg_conmpare_dict[metric]) and not metric_dict[metric]["large_is_better"]:
            model_res += f"& \\textbf{{{num_res:.1f}}}"
        else:
            model_res += f"& {num_res:.1f}"
    model_res += "\\\\\n"
    res += model_res


print(res)


\multirow{2}{*}{\centering \textbf{en}}& Baseline & 4.0& 34.3& 0.6& 8.4& \textbf{100.0}\\
& Ours & \textbf{3.1}& \textbf{34.5}& \textbf{0.4}& \textbf{6.6}& \textbf{100.0}\\
\hline
\multirow{2}{*}{\centering \textbf{ru}}& Baseline & \textbf{0.0}& 0.1& \textbf{0.0}& \textbf{0.0}& 0.5\\
& Ours & 10.2& \textbf{4.4}& 0.2& 2.9& \textbf{100.0}\\
\hline
\multirow{2}{*}{\centering \textbf{de}}& Baseline & \textbf{0.0}& 0.0& \textbf{0.0}& \textbf{0.0}& 0.0\\
& Ours & 5.3& \textbf{23.3}& 0.7& 8.1& \textbf{99.8}\\
\hline
\multirow{2}{*}{\centering \textbf{zh}}& Baseline & 59.9& 1.4& \textbf{0.0}& \textbf{1.4}& 9.7\\
& Ours & \textbf{7.0}& \textbf{23.5}& 0.5& 11.1& \textbf{99.9}\\
\hline
\multirow{2}{*}{\centering \textbf{ja}}& Baseline & 12.3& 14.9& 0.9& \textbf{12.2}& 91.0\\
& Ours & \textbf{10.8}& \textbf{20.4}& \textbf{0.8}& 13.9& \textbf{100.0}\\
\hline
\multirow{2}{*}{\centering \textbf{fr}}& Baseline & \textbf{0.0}& 0.0& \textbf{0.0}& \textbf{0.0}& 0.0\\
& Ours & 9.1& \textbf{30.5}& 1.5& 18.